In [ ]:
#https://www.kaggle.com/datasets/danchyy/nba-mvp-votings-through-history?select=mvp_votings.csv 

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mvp_voting_df = pd.read_csv('./mvp_votings.csv')
test_data_df = pd.read_csv('./test_data.csv')

In [3]:
mvp_voting_df.head()
#test_data_df.head()

,Unnamed: 0,fga,fg3a,fta,per,ts_pct,usg_pct,bpm,season,player,win_pct,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,0,18.6,0.2,6.5,25.1,0.572,28.4,8.0,1980-81,Julius Erving,0.756098,28.0,454.0,690.0,0.658,82.0,35.0,24.6,8.0,4.4,2.1,1.8,0.521,0.222,0.787,13.8,0.231
1,1,18.3,0.9,4.0,19.9,0.528,24.3,5.1,1980-81,Larry Bird,0.756098,20.0,423.0,690.0,0.613,82.0,39.5,21.2,10.9,5.5,2.0,0.8,0.478,0.270,0.863,10.8,0.160
2,2,18.2,0.0,6.9,25.5,0.616,26.3,5.3,1980-81,Kareem Abdul-Jabbar,0.658537,8.0,286.0,690.0,0.414,80.0,37.2,26.2,10.3,3.4,0.7,2.9,0.574,0.000,0.766,14.3,0.230
3,3,19.3,0.0,10.1,25.1,0.585,27.6,3.7,1980-81,Moses Malone,0.487805,8.0,180.0,690.0,0.261,80.0,40.6,27.8,14.8,1.8,1.0,1.9,0.522,0.333,0.757,13.7,0.202
4,4,21.1,0.4,7.6,22.9,0.555,32.3,1.6,1980-81,George Gervin,0.634146,1.0,83.0,690.0,0.120,82.0,33.7,27.1,5.1,3.2,1.1,0.7,0.492,0.257,0.826,10.5,0.182


In [4]:
#Neural network model with 11 features 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = mvp_voting_df[['pts_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'per', 'bpm', 'usg_pct', 'win_pct', 'ws', 'ws_per_48']]
y = mvp_voting_df['award_share']
X_scaled = scaler.fit_transform(X)

In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=11, activation='relu', input_dim = 11))
model.add(tf.keras.layers.Dense(units=22, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 22)                264       
                                                                 
 dense_2 (Dense)             (None, 1)                 23        
                                                                 
Total params: 419 (1.64 KB)
Trainable params: 419 (1.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics = ['mae', 'mse'])

In [8]:
model.fit(X_scaled, y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
16/16 [==============================] - 4s 56ms/step - loss: 0.5058 - mae: 0.4912 - mse: 0.5058 - val_loss: 0.2754 - val_mae: 0.3750 - val_mse: 0.2754
Epoch 2/100
16/16 [==============================] - 0s 16ms/step - loss: 0.2206 - mae: 0.3326 - mse: 0.2206 - val_loss: 0.1466 - val_mae: 0.2720 - val_mse: 0.1466
Epoch 3/100
16/16 [==============================] - 0s 15ms/step - loss: 0.1343 - mae: 0.2621 - mse: 0.1343 - val_loss: 0.0965 - val_mae: 0.2247 - val_mse: 0.0965
Epoch 4/100
16/16 [==============================] - 0s 14ms/step - loss: 0.0950 - mae: 0.2211 - mse: 0.0950 - val_loss: 0.0757 - val_mae: 0.1974 - val_mse: 0.0757
Epoch 5/100
16/16 [==============================] - 0s 13ms/step - loss: 0.0748 - mae: 0.1960 - mse: 0.0748 - val_loss: 0.0642 - val_mae: 0.1831 - val_mse: 0.0642
Epoch 6/100
16/16 [==============================] - 0s 14ms/step - loss: 0.0618 - mae: 0.1772 - mse: 0.0618 - val_loss: 0.0574 - val_mae: 0.1734 - val_mse: 0.0574
Epoch 7/100
16/1

In [9]:
import joblib
model.save('nn_model.h5')  
joblib.dump(scaler, 'nn_scaler.joblib')  

C:\Users\brian\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['nn_scaler.joblib']

In [10]:
#Hyperparameter Tuning
def best_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first units',
                                                    min_value = 1,
                                                    max_value = 11,
                                                    step = 2), activation = activation, input_dim = 11))


    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                    min_value = 1,
                                                    max_value = 22,
                                                    step = 2), activation = activation))
    nn_model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))


    nn_model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse', 'mae'])


    return nn_model

In [11]:
#Kerastuner library
import keras_tuner as kt


tuner = kt.Hyperband(
    best_model,
    objective = 'mse',
    max_epochs = 20,
    hyperband_iterations= 2
)


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [14]:
tuner.search(X_scaled, y, epochs = 20)

Trial 72 Complete [00h 00m 12s]
mse: 0.05969971418380737

Best mse So Far: 0.018799250945448875
Total elapsed time: 00h 01m 56s
INFO:tensorflow:Oracle triggered exit


In [15]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first units': 9,
 'num_layers': 2,
 'units_0': 15,
 'units_1': 17,
 'units_2': 17,
 'units_3': 15,
 'units_4': 7,
 'units_5': 21,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [16]:
tuned_model = tf.keras.models.Sequential()
tuned_model.add(tf.keras.layers.Dense(units=9, activation='relu', input_dim = 11))
tuned_model.add(tf.keras.layers.Dense(units=15, activation='relu'))
tuned_model.add(tf.keras.layers.Dense(units=17, activation='relu'))
tuned_model.add(tf.keras.layers.Dense(units=1, activation='linear'))

tuned_model.summary()

tuned_model.compile(loss='mean_squared_error', optimizer = 'adam', metrics = ['mse', 'mae'])

tuned_model.fit(X_scaled, y, epochs=100, batch_size=32, validation_split=0.2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 9)                 108       
                                                                 
 dense_5 (Dense)             (None, 15)                150       
                                                                 
 dense_6 (Dense)             (None, 17)                272       
                                                                 
 dense_7 (Dense)             (None, 1)                 18        
                                                                 
Total params: 548 (2.14 KB)
Trainable params: 548 (2.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
16/16 [==============================] - 4s 44ms/step - loss: 0.1412 - mse: 0.1412 - mae: 0.2766 - val_loss: 0.1205 - val_mse: 0.1205 - val_mae:

In [17]:
tuned_model.save('tuned_nn_model.h5')  

C:\Users\brian\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
#Linear Model 
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_X = mvp_voting_df.copy()
lr_X.drop(columns=['Unnamed: 0', 'player', 'season', 'votes_first', 'points_won', 'points_max', 'award_share'], axis=1, inplace=True)
lr_y = mvp_voting_df['award_share']

In [19]:
#Linear Model Feature Selection 
from sklearn.feature_selection import SelectKBest, f_regression
fs = SelectKBest(score_func=f_regression, k='all')
fs.fit(lr_X,lr_y)
lr_X_fs = fs.transform(lr_X)
feature_names = fs.get_feature_names_out()
fs_df = pd.DataFrame({'feature_names': feature_names, 'feature_scores': fs.scores_})
fs_df.sort_values(by = 'feature_scores', ascending=False)

,feature_names,feature_scores
18,ws,417.928709
19,ws_per_48,354.117235
3,per,347.542067
6,bpm,317.003508
10,pts_per_g,144.043844
2,fta,93.138309
7,win_pct,92.627672
5,usg_pct,91.893246
0,fga,73.265688
4,ts_pct,40.267982


In [20]:
#Linear Model w/ top 7 features
updated_lr_X = lr_X[['ws', 'ws_per_48','per', 'bpm', 'pts_per_g', 'win_pct', 'usg_pct']]
lr_scaler = StandardScaler()
updated_lr_X_scaled = lr_scaler.fit_transform(updated_lr_X)
lr_model.fit(updated_lr_X_scaled, lr_y)
joblib.dump(lr_model, 'lr_model.joblib')  
joblib.dump(lr_scaler, 'lr_scaler.joblib')  

['lr_scaler.joblib']

In [21]:
#Random Forest Model 
from sklearn.ensemble import RandomForestRegressor
rf_X = mvp_voting_df.copy()
rf_X.drop(columns=['Unnamed: 0', 'player', 'season', 'votes_first', 'points_won', 'points_max', 'award_share'], axis=1, inplace=True)
rf_y = mvp_voting_df['award_share']

In [22]:
rf_scaler = StandardScaler()
rf_X_scaled = rf_scaler.fit_transform(rf_X)
rf_model = RandomForestRegressor(n_estimators=500, random_state=78)
rf_model.fit(rf_X_scaled, rf_y)
feature_importance = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, rf_X.columns), reverse=True)

[(0.3917624990923945, 'ws'),
 (0.1273102273962831, 'win_pct'),
 (0.09112543451046652, 'per'),
 (0.05778553826528482, 'bpm'),
 (0.043540489517779744, 'ws_per_48'),
 (0.03535663053947361, 'usg_pct'),
 (0.03348384340760627, 'fga'),
 (0.02527649055226881, 'ft_pct'),
 (0.022963301010964755, 'ast_per_g'),
 (0.022407436306942752, 'mp_per_g'),
 (0.02118451376981331, 'fta'),
 (0.021113093425223765, 'fg_pct'),
 (0.01763241842930172, 'pts_per_g'),
 (0.015601249034509917, 'ts_pct'),
 (0.015314203172930849, 'trb_per_g'),
 (0.01438073761509134, 'fg3_pct'),
 (0.013528987256828585, 'g'),
 (0.010963354597181086, 'blk_per_g'),
 (0.009913322547373418, 'fg3a'),
 (0.009356229552281103, 'stl_per_g')]

In [23]:
#Updated model using feature importance with 6 features from nn_model
updated_rf_X = rf_X[['per', 'bpm', 'usg_pct', 'win_pct', 'ws', 'ws_per_48']]
updated_rf_X_scaled = rf_scaler.fit_transform(updated_rf_X)
updated_rf_model = RandomForestRegressor(n_estimators=500, random_state=78)
updated_rf_model.fit(updated_rf_X_scaled, rf_y)
joblib.dump(updated_rf_model, 'rf_model.joblib')
joblib.dump(rf_scaler, 'rf_scaler.joblib')

['rf_scaler.joblib']